In [ ]:
!pip install -q mljar-supervised

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 135.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.6/386.6 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 22.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [ ]:
# 1. Library Imports
import pandas as pd
from supervised.automl import AutoML
import matplotlib.pyplot as plt

In [ ]:
import sklearn

In [ ]:
# 2. Load Dataset
df = pd.read_csv("./data.csv")  # Kaggle house price dataset

# 3. Feature Engineering (Minimal)
df = df.drop(columns=["date", "street", "city", "statezip", "country"], errors="ignore")  # remove text
df = df.dropna()  # simple NaN removal
df = df[df['price'] > 0]  # remove zero prices

# 4. Define Features and Target
X = df.drop(columns=['price'])
y = df['price']

# 5. Train/Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 6. Run MLJAR AutoML
automl = AutoML(
    mode="Compete",  # 'Explain', 'Perform', 'Compete', 'Optuna'
    total_time_limit=300,
    eval_metric="mae",
    algorithms=["Xgboost", "Random Forest", "Linear", "Extra Trees", "CatBoost"],  # at least 3 models
    train_ensemble=True,
    verbose=1
)

automl.fit(X_train, y_train)

AutoML directory: AutoML_1
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Xgboost', 'Random Forest', 'Linear', 'Extra Trees', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 181644.368779 trained in 0.33 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 5-fold CV Shuffle
* Step simple_algorithms will try to check up to 1 model
1_Linear mae 167018.019132 trained in 1.51 seconds
* Step default_algorithms will try to check up to 4 models
2_Default_Xgboost mae 176194.761366 trained in 1.86 seconds
3_Default_CatBoost mae 164639.042037 trained in 2.78 seconds
4_De

AutoML(algorithms=['Xgboost', 'Random Forest', 'Linear', 'Extra Trees',
                   'CatBoost'],
       eval_metric='mae', mode='Compete', total_time_limit=300)

In [ ]:
# 7. Evaluate on Test Set
preds = automl.predict(X_test)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, preds)
#rmse = mean_squared_error(y_test, preds, squared=False)
rmse = mean_squared_error(y_test, preds) ** 0.5
r2 = r2_score(y_test, preds)

print(f"\n🔎 Test Performance:")
print(f"MAE : {mae:,.0f}")
print(f"RMSE: {rmse:,.0f}")
print(f"R²   : {r2:.3f}")


🔎 Test Performance:
MAE : 148,520
RMSE: 243,809
R²   : 0.600


In [ ]:
# 8. View AutoML Report (locally saved)
import os
print("\n📝 MLJAR Report saved in:", automl._results_path)


📝 MLJAR Report saved in: AutoML_1
